### Top k most traded stocks (by volume)  

In [1]:
from typing import List, Tuple
import heapq
from sortedcontainers import SortedDict # https://grantjenks.com/docs/sortedcontainers/sorteddict.html

In [2]:
class Stock:
    
    def __init__(self, name: str=None, volume: int=0):
        self.name = name
        self.volume = volume

In [3]:
class Trades:
    
    def __init__(self) -> None:
        self.stocks = {} # name -> volume
        self.sortedStocks = SortedDict() # -volume -> List[name]
        
    def insert(self, name: str, volume: int) -> None:
        # time = O(logN) search BST
        oldVolume = self.stocks.get(name, 0)
        newVolume = oldVolume + volume
        self.stocks[name] = newVolume
        if -oldVolume in self.sortedStocks:
            if self.sortedStocks[-oldVolume] == set([name]):
                del self.sortedStocks[-oldVolume]
            else:
                self.sortedStocks[-oldVolume].remove(name)
        if -newVolume in self.sortedStocks:
            self.sortedStocks[-newVolume].add(name)
        else:
            self.sortedStocks[-newVolume] = set([name])

    def getMost(self, k: int) -> List[Tuple[str, int]]:
        # time = O(K)
        count = 0
        res = []
        for volume, names in self.sortedStocks.items():
            for name in names:
                res.append((name, -volume))
                count += 1
                if count == k:
                    break
            if count == k:
                break
        return res

    def getMost2(self, k: int) -> List[Tuple[str, int]]:
        # time = O(N.logK)
        return heapq.nlargest(k, self.stocks.items(), key=lambda item: (item[1], item[0]))

In [4]:
sortedDict = SortedDict()

sortedDict[-1] = set(['a'])
sortedDict[-1].add('b')
sortedDict[-1].add('c')

sortedDict[-2] = set(['x'])
sortedDict[-2].add('y')

print(sortedDict)

sortedDict[-1].remove('c')

print(sortedDict)

del sortedDict[-1]

print(sortedDict)

SortedDict({-2: {'x', 'y'}, -1: {'c', 'a', 'b'}})
SortedDict({-2: {'x', 'y'}, -1: {'a', 'b'}})
SortedDict({-2: {'x', 'y'}})


In [5]:
trades = Trades()

trades.insert('AAPL', 100)
trades.insert('MSFT', 100)
trades.insert('AAPL', 200)
trades.insert('AMZN', 250)
trades.insert('AMAT', 300)
trades.insert('MSFT', 100)

In [6]:
stocks = trades.getMost(2)
print(stocks)

[('AMAT', 300), ('AAPL', 300)]


In [7]:
stocks2 = trades.getMost2(2)
print(stocks2)

[('AMAT', 300), ('AAPL', 300)]


In [8]:
from sortedcontainers import SortedDict

class Leaderboard:
    # using a SortedDict
    
    def __init__(self):
        self.scores = {} # playerId -> score
        self.sortedScores = SortedDict() # -score -> num of players having this score, Binary Search Tree, small to large

    def addScore(self, playerId: int, score: int) -> None:
        # time = O(logN) search BST
        if playerId not in self.scores:
            self.scores[playerId] = score
            self.sortedScores[-score] = self.sortedScores.get(-score, 0) + 1
        else:
            oldScore = self.scores[playerId]
            if self.sortedScores[-oldScore] == 1:
                del self.sortedScores[-oldScore]
            else:
                self.sortedScores[-oldScore] -= 1
            newScore = oldScore + score
            self.scores[playerId] = newScore
            self.sortedScores[-newScore] = self.sortedScores.get(-newScore, 0) + 1

    def top(self, K: int) -> int:
        # time = O(K)
        count, total = 0, 0
        for score, num_players in self.sortedScores.items():
            for _ in range(num_players):
                total += -score
                count += 1
                if count == K:
                    break
            if count == K:
                break
        return total

    def reset(self, playerId: int) -> None:
        # time = O(logN) search BST
        oldScore = self.scores[playerId]
        if self.sortedScores[-oldScore] == 1:
            del self.sortedScores[-oldScore]
        else:
            self.sortedScores[-oldScore] -= 1
        del self.scores[playerId]

# Your Leaderboard object will be instantiated and called as such:
# obj = Leaderboard()
# obj.addScore(playerId,score)
# param_2 = obj.top(K)
# obj.reset(playerId)